# Clase 5



## Contenidos
--------------

1. Bases de datos de redes metabólicas
2. Definición de restricciones y variables adicionales
3. Ajustar modelo a variables experimentales

# Bases de datos de redes metabólicas

Hoy exploraremos el repositorio de modelos [BiGG](http://bigg.ucsd.edu/). 


Ejercicio
------------

Descarga el modelo de *Bacillus subtilis* iYO844 en formato SBML.

In [ ]:
import cobra
# Cargamos el modelo de Bacillus subtilis
bacilusModel=cobra.io.read_sbml_model("iYO844.xml")
%pwd
#%ls -l

In [ ]:
# Información general del modelo
bacilusModel

In [ ]:
# Reacciones de consumo de sustratos desde el medio de cultivo
bacilusModel.medium

In [ ]:
# Simular los flujos con FBA
solution=bacilusModel.optimize()

In [ ]:
# Resumen de la solucion
bacilusModel.summary()

## Especificando restricciones adicionales


Supón que queremos asegurarnos que dos reaccioens tengan el mismo flujo en nuestro modelo. Podemos agregar este criterio como una restricción a nuestro modelo a través de la siguiente sintaxis:

``` python
same_flux = model.problem.Constraint(
    bacilusModel.reactions.FBA.flux_expression - model.reactions.NH4t.flux_expression,
    lb=0,
    ub=0)
bacilusModel.add_cons_vars(same_flux)
```

In [ ]:
bacilusModel=cobra.io.read_sbml_model("iYO844.xml")

same_flux = bacilusModel.problem.Constraint(
    3*bacilusModel.reactions.EX_glc__D_e.flux_expression - 
    bacilusModel.reactions.EX_o2_e.flux_expression,
    lb=0,
    ub=0)
bacilusModel.add_cons_vars(same_flux)
solution=bacilusModel.optimize()
bacilusModel.summary()

## Ejercicio


En el modelo de *Bacillus subtilis* especifica que el consumo de oxygeno 

1. sea igual al consumo de glucosa. 
2. sea 3 veces el consumo de glucosa

Especificando variables adicionales
---------------------------------------

Supón que queremos medir las diferencia entre el flujo de dos reacciones. Para esto podemos crear una variable adicional ```diff```.


```python
difference = model.problem.Variable('difference')
```

y luego definirla como la diferencia entre las dos reacciones de interes, por ejemplo:

```python
constraint = model.problem.Constraint(
    model.reactions.EX_glc__D_e.flux_expression -
    model.reactions.EX_nh4_e.flux_expression - difference,
    lb=0,
    ub=0)
model.add_cons_vars([difference, constraint])
```

Ejercicio
------------

En el modelo de *Bacillus subtilis* define una varibale que mida la diferencia entre los flujos de consumo de glucosa y nh4_e. 

Medir calidad de las simulaciones
--------------------------------------

Si conocemos el valor de uno o varios flujos metabólicos podemos ajustar nuestro modelo a estos datos. Hay varias maneras de hacer esto, la primera es simplemente fijar el valor de los flujos como veremos a continuación. 

A modo de ejemplo usaremos *Saccharomyces cerevisae* iMM904 [(descargar aquí)](http://bigg.ucsd.edu/models/iMM904) y los flujos experimentales compilados por [García et al 2012](https://journals.plos.org/plosone/article?id=10.1371%2Fjournal.pone.0043006)


In [ ]:
# Let's import the yeast model
import cobra
model = cobra.io.read_sbml_model("iMM904.xml")

In [ ]:
# And check the solution using default parameters
solution=model.optimize()
model.summary()

In [ ]:
# Now, let's modify the ethanol flux
# First, lets find the reactions associated with ethanol
import re

def buscarMetabolito(nombreMetabolite):
    metabolites=[]
    for metabolite in model.metabolites:
        if re.match(nombreMetabolite,metabolite.name,re.IGNORECASE):# and re.match("_e",metabolite.name): 
            metabolites.append(metabolite)
    if len(metabolites)==0:
        print( "Not found metabolite" )
        return None
    else: #print "2 lets find the reactions associated with the name"
        for metabolite in metabolites:
            print( "==================================")
            print( metabolite.name, metabolite.id)
            print( "==================================")
            for reaction in model.metabolites.get_by_id(metabolite.id).reactions:
                print( reaction )
result=buscarMetabolito("D-Glucose")

Es posible que errores de medición resulten en datos de flujos metabólicos que no son estequiometricamente compatibles. Para estos casos, una alternativa es fijar un rango de valores en lugar de un valor exacto. 

Ejercicio 1
---------

Usando los valores experimentales compilados por [García et al 2012](https://journals.plos.org/plosone/article?id=10.1371%2Fjournal.pone.0043006) desde la referencia [20] (ver la columna **References** en Table_S1.doc):

1. Fija la reacción de consumo de glucosa, succinato, ethanol, y piruvato a datos experimentales (haciendo 1 esto debería ser trivial).